In [60]:
from langchain_community.document_loaders import TextLoader
from langchain_text_splitters import CharacterTextSplitter
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS

docs_convert  = TextLoader("speech.txt")

text_load = docs_convert.load()
text_load



[Document(metadata={'source': 'speech.txt'}, page_content='Democracy and Zulifqar Ali Bhutto (Ex Prime minister of Pakistan)\n\nDemocracy is our Politics, Islam is our religion, Socialism is our Economy.\n\n The second elected Prime Minister of pakistan was Banzeer Bhutto daughter of zulifqar Ali Bhutto. \n \n\n This statement was dileverd by Ex Pakistani Prime Minister Zulifqar Ali Bhutto. \n\n\n He was the first elected Prime minister of Pakistan. \n \n He become a prime minister of Pakistan after winning first general elections of Pakistan.\n')]

In [61]:
splitter = CharacterTextSplitter(chunk_size= 200, chunk_overlap= 20)
text_splitter = splitter.split_documents(text_load)
text_splitter

[Document(metadata={'source': 'speech.txt'}, page_content='Democracy and Zulifqar Ali Bhutto (Ex Prime minister of Pakistan)\n\nDemocracy is our Politics, Islam is our religion, Socialism is our Economy.'),
 Document(metadata={'source': 'speech.txt'}, page_content='The second elected Prime Minister of pakistan was Banzeer Bhutto daughter of zulifqar Ali Bhutto. \n \n\n This statement was dileverd by Ex Pakistani Prime Minister Zulifqar Ali Bhutto.'),
 Document(metadata={'source': 'speech.txt'}, page_content='He was the first elected Prime minister of Pakistan. \n \n He become a prime minister of Pakistan after winning first general elections of Pakistan.')]

In [62]:
embeddings = HuggingFaceEmbeddings(model_name= "all-MiniLM-L6-v2")
db = FAISS.from_documents(text_splitter, embeddings)
db

In [63]:
#Quering from our Vector Store data Base
query = "First Elected Prime Minister of Pakistan"
answer= db.similarity_search(query)
answer

[Document(metadata={'source': 'speech.txt'}, page_content='He was the first elected Prime minister of Pakistan. \n \n He become a prime minister of Pakistan after winning first general elections of Pakistan.'),
 Document(metadata={'source': 'speech.txt'}, page_content='The second elected Prime Minister of pakistan was Banzeer Bhutto daughter of zulifqar Ali Bhutto. \n \n\n This statement was dileverd by Ex Pakistani Prime Minister Zulifqar Ali Bhutto.'),
 Document(metadata={'source': 'speech.txt'}, page_content='Democracy and Zulifqar Ali Bhutto (Ex Prime minister of Pakistan)\n\nDemocracy is our Politics, Islam is our religion, Socialism is our Economy.')]

In [64]:
answer[0].page_content

'He was the first elected Prime minister of Pakistan. \n \n He become a prime minister of Pakistan after winning first general elections of Pakistan.'

In [65]:
retreiver = db.as_retriever()
docs = retreiver.invoke(query)
docs

[Document(metadata={'source': 'speech.txt'}, page_content='He was the first elected Prime minister of Pakistan. \n \n He become a prime minister of Pakistan after winning first general elections of Pakistan.'),
 Document(metadata={'source': 'speech.txt'}, page_content='The second elected Prime Minister of pakistan was Banzeer Bhutto daughter of zulifqar Ali Bhutto. \n \n\n This statement was dileverd by Ex Pakistani Prime Minister Zulifqar Ali Bhutto.'),
 Document(metadata={'source': 'speech.txt'}, page_content='Democracy and Zulifqar Ali Bhutto (Ex Prime minister of Pakistan)\n\nDemocracy is our Politics, Islam is our religion, Socialism is our Economy.')]

In [66]:
docs_1 = db.similarity_search_with_score(query)
docs_1

[(Document(metadata={'source': 'speech.txt'}, page_content='He was the first elected Prime minister of Pakistan. \n \n He become a prime minister of Pakistan after winning first general elections of Pakistan.'),
  0.21267918),
 (Document(metadata={'source': 'speech.txt'}, page_content='The second elected Prime Minister of pakistan was Banzeer Bhutto daughter of zulifqar Ali Bhutto. \n \n\n This statement was dileverd by Ex Pakistani Prime Minister Zulifqar Ali Bhutto.'),
  0.85097474),
 (Document(metadata={'source': 'speech.txt'}, page_content='Democracy and Zulifqar Ali Bhutto (Ex Prime minister of Pakistan)\n\nDemocracy is our Politics, Islam is our religion, Socialism is our Economy.'),
  1.1780195)]

In [67]:
embed_query = embeddings.embed_query(query)
embed_query

[0.015181386843323708,
 0.06688741594552994,
 0.0011891628382727504,
 0.08437973260879517,
 -0.014389305375516415,
 -0.043055128306150436,
 0.058757442981004715,
 -0.0016707098111510277,
 -0.05405876785516739,
 0.01788567565381527,
 0.006664360407739878,
 -0.013701382093131542,
 0.13836266100406647,
 0.07173889130353928,
 0.039259638637304306,
 0.0074163381941616535,
 -0.03185376897454262,
 0.02810324914753437,
 0.04873998835682869,
 -0.033328671008348465,
 0.010641658678650856,
 0.05169166997075081,
 0.006462117191404104,
 -0.03941517695784569,
 -0.010149543173611164,
 -0.04688180983066559,
 0.022867262363433838,
 0.027623288333415985,
 -0.03017735667526722,
 -0.021016789600253105,
 -0.009896150790154934,
 -0.1178707703948021,
 0.0892813578248024,
 0.02992066740989685,
 0.08471250534057617,
 0.022976644337177277,
 -0.031906191259622574,
 0.09246663004159927,
 -0.05429733172059059,
 -0.031743016093969345,
 0.047981634736061096,
 -0.033440001308918,
 0.04100404679775238,
 -0.01353147998

In [68]:
len(embed_query)

384

In [69]:
answer_1 =  db.similarity_search_by_vector(embed_query)
answer_1

[Document(metadata={'source': 'speech.txt'}, page_content='He was the first elected Prime minister of Pakistan. \n \n He become a prime minister of Pakistan after winning first general elections of Pakistan.'),
 Document(metadata={'source': 'speech.txt'}, page_content='The second elected Prime Minister of pakistan was Banzeer Bhutto daughter of zulifqar Ali Bhutto. \n \n\n This statement was dileverd by Ex Pakistani Prime Minister Zulifqar Ali Bhutto.'),
 Document(metadata={'source': 'speech.txt'}, page_content='Democracy and Zulifqar Ali Bhutto (Ex Prime minister of Pakistan)\n\nDemocracy is our Politics, Islam is our religion, Socialism is our Economy.')]

In [70]:
db.save_local("faiss_index")

In [79]:
new_db = FAISS.load_local("faiss_index", embeddings, allow_dangerous_deserialization=True)
docs= new_db.similarity_search(query)
docs

[Document(metadata={'source': 'speech.txt'}, page_content='He was the first elected Prime minister of Pakistan. \n \n He become a prime minister of Pakistan after winning first general elections of Pakistan.'),
 Document(metadata={'source': 'speech.txt'}, page_content='The second elected Prime Minister of pakistan was Banzeer Bhutto daughter of zulifqar Ali Bhutto. \n \n\n This statement was dileverd by Ex Pakistani Prime Minister Zulifqar Ali Bhutto.'),
 Document(metadata={'source': 'speech.txt'}, page_content='Democracy and Zulifqar Ali Bhutto (Ex Prime minister of Pakistan)\n\nDemocracy is our Politics, Islam is our religion, Socialism is our Economy.')]